In [1]:
%matplotlib inline

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import patsy
import statsmodels.api as sm
import scipy.stats as stats
from scipy.stats import ttest_ind, chisquare, normaltest


In [2]:
###CSGO Most viewed major tournament per year data importing

In [139]:
#Import csvs
majors = pd.read_csv("ESL One Cologne 2015 - SullyGnome.csv")
df1 = pd.read_csv("MLG Major Championship Columbus 2016 - SullyGnome.csv")
df2 = pd.read_csv("ELEAGUE Major Atlanta 2017 - SullyGnome.csv")
df3 = pd.read_csv("ELEAGUE Major Boston 2018 - SullyGnome.csv")
print(majors)
print(df1)
print(df2)
print(df3)

    Unnamed: 0                Stream start time  Stream length  Avg viewers  \
0            1    Sunday 23rd August 2015 20:00          24585         3495   
1            2  Saturday 22nd August 2015 08:45           2100       343613   
2            3    Friday 21st August 2015 09:45           1350       265423   
3            4  Thursday 20th August 2015 21:30            720       104183   
4            5  Thursday 20th August 2015 11:45            570       446602   
5            6  Thursday 20th August 2015 04:30            420        39202   
6            7  Thursday 20th August 2015 03:45             30         9317   
7            8  Thursday 13th August 2015 09:15           9735         6967   
8            9   Tuesday 11th August 2015 18:00           2100         1884   
9           10     Sunday 9th August 2015 16:00           2970         2305   
10          11     Sunday 9th August 2015 08:15            420         8431   
11          12   Saturday 8th August 2015 14:00     

In [140]:
#Drop non-tournament rows
majors = majors.drop([7,21])
df2 = df2.drop([11, 12])

In [141]:
#aggregate
majors = majors.append(df1)
majors = majors.append(df2)
majors = majors.append(df3)
majors

,Unnamed: 0,Stream start time,Stream length,Avg viewers,Peak viewers,Followers gained,Followers per hour,Views,Views per hour,Games
0,1,Sunday 23rd August 2015 20:00,24585,3495,127803,443,1.08,992400,2421.96,Counter-Strike: Global Offensive|Counter-Strik...
1,2,Saturday 22nd August 2015 08:45,2100,343613,756162,115811,3308.88,6100587,174302.48,Counter-Strike: Global Offensive|Counter-Strik...
2,3,Friday 21st August 2015 09:45,1350,265423,567269,65429,2907.96,11336096,503826.48,Counter-Strike: Global Offensive|Counter-Strik...
3,4,Thursday 20th August 2015 21:30,720,104183,517148,25822,2151.84,4962773,413564.40,Counter-Strike: Global Offensive|Counter-Strik...
4,5,Thursday 20th August 2015 11:45,570,446602,591641,111068,11691.36,10676674,1123860.44,Counter-Strike: Global Offensive|Counter-Strik...
5,6,Thursday 20th August 2015 04:30,420,39202,135618,21805,3115.00,379211,54173.00,Counter-Strike: Global Offensive|Counter-Strik...
6,7,Thursday 20th August 2015 03:45,30,9317,13142,697,1394.00,13081,26162.00,Counter-Strike: Global Offensive|Counter-Strik...
8,9,Tuesday 11th August 2015 18:00,2100,1884,11484,977,27.92,52891,1511.16,Counter-Strike: Global Offensive|Counter-Strik...
9,10,Sunday 9th August 2015 16:00,2970,2305,4775,1906,38.52,82481,1666.28,Counter-Strike: Global Offensive|Counter-Strik...
10,11,Sunday 9th August 2015 08:15,420,8431,12999,563,80.44,40953,5850.44,Counter-Strike: Global Offensive|Counter-Strik...


In [142]:
#Add new index and remove unused columns
majors = majors.drop(columns=['Unnamed: 0', 'Followers per hour', 'Followers gained', 'Games'], axis = 1)
majors

,Stream start time,Stream length,Avg viewers,Peak viewers,Views,Views per hour
0,Sunday 23rd August 2015 20:00,24585,3495,127803,992400,2421.96
1,Saturday 22nd August 2015 08:45,2100,343613,756162,6100587,174302.48
2,Friday 21st August 2015 09:45,1350,265423,567269,11336096,503826.48
3,Thursday 20th August 2015 21:30,720,104183,517148,4962773,413564.40
4,Thursday 20th August 2015 11:45,570,446602,591641,10676674,1123860.44
5,Thursday 20th August 2015 04:30,420,39202,135618,379211,54173.00
6,Thursday 20th August 2015 03:45,30,9317,13142,13081,26162.00
8,Tuesday 11th August 2015 18:00,2100,1884,11484,52891,1511.16
9,Sunday 9th August 2015 16:00,2970,2305,4775,82481,1666.28
10,Sunday 9th August 2015 08:15,420,8431,12999,40953,5850.44


In [143]:
#Changing stream length to a unit in hours instead of minutes
def change_hours(inValue):
    return inValue / 60

majors['Stream length'] = majors['Stream length'].apply(change_hours)
majors

,Stream start time,Stream length,Avg viewers,Peak viewers,Views,Views per hour
0,Sunday 23rd August 2015 20:00,409.75,3495,127803,992400,2421.96
1,Saturday 22nd August 2015 08:45,35.00,343613,756162,6100587,174302.48
2,Friday 21st August 2015 09:45,22.50,265423,567269,11336096,503826.48
3,Thursday 20th August 2015 21:30,12.00,104183,517148,4962773,413564.40
4,Thursday 20th August 2015 11:45,9.50,446602,591641,10676674,1123860.44
5,Thursday 20th August 2015 04:30,7.00,39202,135618,379211,54173.00
6,Thursday 20th August 2015 03:45,0.50,9317,13142,13081,26162.00
8,Tuesday 11th August 2015 18:00,35.00,1884,11484,52891,1511.16
9,Sunday 9th August 2015 16:00,49.50,2305,4775,82481,1666.28
10,Sunday 9th August 2015 08:15,7.00,8431,12999,40953,5850.44


In [144]:
# dropped the outliers where views per hour = 0 
majors = majors[(majors['Views per hour'] > 0)]
majors

,Stream start time,Stream length,Avg viewers,Peak viewers,Views,Views per hour
0,Sunday 23rd August 2015 20:00,409.75,3495,127803,992400,2421.96
1,Saturday 22nd August 2015 08:45,35.00,343613,756162,6100587,174302.48
2,Friday 21st August 2015 09:45,22.50,265423,567269,11336096,503826.48
3,Thursday 20th August 2015 21:30,12.00,104183,517148,4962773,413564.40
4,Thursday 20th August 2015 11:45,9.50,446602,591641,10676674,1123860.44
5,Thursday 20th August 2015 04:30,7.00,39202,135618,379211,54173.00
6,Thursday 20th August 2015 03:45,0.50,9317,13142,13081,26162.00
8,Tuesday 11th August 2015 18:00,35.00,1884,11484,52891,1511.16
9,Sunday 9th August 2015 16:00,49.50,2305,4775,82481,1666.28
10,Sunday 9th August 2015 08:15,7.00,8431,12999,40953,5850.44


In [145]:
#formatting time after peter's and sorting
majors['Stream start time'] = pd.to_datetime(majors['Stream start time'])
majors = majors.sort_values(['Stream start time'])
majors = majors.reset_index(drop='True')
majors

C:\Users\xluto\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


,Stream start time,Stream length,Avg viewers,Peak viewers,Views,Views per hour
0,2015-08-02 23:45:00,6.50,10150,53756,102756,15808.60
1,2015-08-03 07:45:00,6.25,8371,14342,36100,5776.00
2,2015-08-03 14:15:00,41.50,1815,4504,102808,2477.32
3,2015-08-05 08:00:00,7.00,44444,76294,312108,44586.84
4,2015-08-05 15:15:00,1.25,60229,90547,166241,132992.80
5,2015-08-05 16:45:00,7.25,17584,90547,227584,31390.88
6,2015-08-06 07:00:00,0.75,6171,10537,9513,12684.00
7,2015-08-06 08:00:00,30.50,35263,92800,963738,31597.96
8,2015-08-07 14:45:00,23.00,28679,107891,544930,23692.60
9,2015-08-08 14:00:00,18.00,5960,15945,74950,4163.88


In [146]:
#Printing out to file.
majors.to_csv('csgo - aggregate data.csv', index=None, header=True)